# Import Statements

In [8]:
from scipy.stats import norm, gamma
import time, datetime
import numpy as np, pandas as pd
from os.path import abspath, dirname, join
import pyodbc

import importlib.util
spec = importlib.util.spec_from_file_location("useful_func", r"C:\Users\armando_borjas\Documents\Python\Calculators\general_POE.py")
POE = importlib.util.module_from_spec(spec)
spec.loader.exec_module(POE)

pd.set_option("display.max_columns",500)

# Functions 

In [2]:
def sqlserver_sql(q, server = 'SQL2012', db = 'TMC_IRASV6_STAGE'):
    driver = '{SQL Server Native Client 11.0}'
    conn = pyodbc.connect("Driver="+driver+";Server="+server+";Database="+db+";Trusted_Connection=yes;")

    # performing query to database
    df = pd.read_sql_query(q,conn)

    conn.close()
    return df

In [3]:
def get_ili_ranges(line):
    q1 = f"""set nocount on;
            select ll.LineName, ld.code [status], r.* from InlineInspectionRange r
            join StationSeries ss on r.BeginStationSeriesId = ss.Id
            join LineLoop ll on ss.LineLoopId = ll.Id
            join ListDomain ld on r.ILIRStatusDomainId = ld.Id
            where ll.LineName like '%{line}%'
            order by r.ILIRStartDate desc"""

    return sqlserver_sql(q1)

In [44]:
def get_features_for_poe(ILIRID, start=1, amt=999999999):
    query = f"""select
            c.RN,
            ll.linename [line],

            ld4.code [vendor], 

            format(r.ILIRStartDate,'yyyy-MM-dd') [ILIRStartDate],
            ld3.code [tool],

            f.ILIFFeatureNumber [FeatureID],
            ld.code [status], 
            ld2.code [type],
            (f.StationNum*mlv.MultiplierNum+mlv.FactorNum) [chainage_m], 
            f.ILIFSurfaceInd, 
            f.ILIFPeakDepthPct [depth_fraction], 
            f.ILIFLength [length_mm], 
            f.ILIFWidth [width_mm],
            f.GrowthRate [vendor_cgr_mmpyr],
            f.GrowthRateStdDev [vendor_cgr_sd],

            format(a.PipeInserviceDate,'yyyy-MM-dd') [install_date],
            a.PipeOutsideDiameter [OD_inch],
            a.PipeWallThickness [WT_mm],
            a.PipeGrade [grade_MPa],
            a.PipeToughness [toughness_J],
            a.[begin_ps_c],
            a.[end_ps_c],

            b.MAOP_kPa,
            b.begin_maop_c,
            b.end_maop_c  

            from InlineInspectionFeature f

            left join ListDomain ld on f.ILIFStatusDomainId = ld.Id
            left join ListDomain ld2 on f.ILIFTypeDomainId = ld2.Id
            left join StationSeries ss on f.StationSeriesId = ss.id
            left join LineLoop ll on ss.LineLoopId = ll.Id
            left join inlineinspectionrange r on f.inlineinspectionrangeid = r.id
            left join ListDomain ld3 on r.ILIRToolDomainId = ld3.Id
            left join InlineInspection i on f.InlineInspectionId = i.Id
            left join ListDomain ld4 on i.ILICompanyDomainId = ld4.Id
            left join MLVCorrection mlv on f.StationSeriesId = mlv.StationSeriesId

            left join 
                (select ll.id [LinloopId],
                ll.LineName,
                ps.EffectiveStartDate,
                ps.PipeInserviceDate,
                ps.PipeOutsideDiameter,
                ps.PipeWallThickness,
                ps.PipeGrade,
                ps.PipeToughness,
                (ps.BeginStationNum*mlv1.MultiplierNum+mlv1.FactorNum) [begin_ps_c],
                (ps.EndStationNum*mlv2.MultiplierNum+mlv2.FactorNum) [end_ps_c]
                from PipeSegment ps
                join StationSeries ss on ps.BeginStationSeriesId = ss.id
                join LineLoop ll on ss.LineLoopId = ll.Id
                join MLVCorrection mlv1 on ps.BeginStationSeriesId = mlv1.StationSeriesId
                join MLVCorrection mlv2 on ps.EndStationSeriesId = mlv2.StationSeriesId
                where ps.EffectiveEndDate is null
                ) a on ((f.StationNum*mlv.MultiplierNum+mlv.FactorNum) between a.[begin_ps_c] and a.[end_ps_c]) and a.LinloopId = ll.id

            left join 
                (select ll.id [LinloopId],
                ll.LineName,
                maop.EffectiveEndDate,
                maop.BeginStationSeriesId,
                maop.MaxAllowablePressure [MAOP_kPa],
                maop.BeginStationNum*mlv1.MultiplierNum+mlv1.FactorNum [begin_maop_c],
                maop.EndStationNum*mlv2.MultiplierNum+mlv2.FactorNum [end_maop_c]
                from maop maop
                join StationSeries ss on ss.id = maop.BeginStationSeriesId
                join LineLoop ll on ss.LineLoopId = ll.Id
                join MLVCorrection mlv1 on maop.BeginStationSeriesId = mlv1.StationSeriesId
                join MLVCorrection mlv2 on maop.EndStationSeriesId = mlv2.StationSeriesId
                where maop.EffectiveEndDate is null
                ) b on ((f.StationNum*mlv.MultiplierNum+mlv.FactorNum) between b.[begin_maop_c] and b.[end_maop_c]) and b.[LinloopId] = ll.id 

            left join
                (select ROW_NUMBER() over(partition by ff.InlineInspectionRangeId order by ff.id asc)  [RN],
                ff.id,
                ff.InlineInspectionRangeId
                from InlineInspectionFeature ff
                )  c on f.Id = c.Id and f.InlineInspectionRangeId = c.InlineInspectionRangeId

            where f.InlineInspectionRangeId = {ILIRID} and (c.RN between {start} and {start+amt})
            order by chainage_m asc"""
    
    return sqlserver_sql(query)

In [5]:
def get_cracks_for_poe(ILIRID, start=1, amt=999999999):
    query = f"""select
            c.RN,
            ll.linename [line],

            ld4.code [vendor], 

            format(r.ILIRStartDate,'yyyy-MM-dd') [ILIRStartDate],
            ld3.code [tool],

            f.ILICAAnomalyNumber [FeatureID],
            ld.code [status], 
            ld2.code [type],
            (f.StationNum*mlv.MultiplierNum+mlv.FactorNum) [chainage_m], 
            f.ILICASurfaceInd, 
            f.ILICADepthPct [depth_fraction], 
            f.ILICALength [length_mm], 
            f.ILICAWidth [width_mm], 

            format(a.PipeInserviceDate,'yyyy-MM-dd') [install_date],
            a.PipeOutsideDiameter [OD_inch],
            a.PipeWallThickness [WT_mm],
            a.PipeGrade [grade_MPa],
            a.PipeToughness [toughness_J],
            a.[begin_ps_c],
            a.[end_ps_c],

            b.MAOP_kPa,
            b.begin_maop_c,
            b.end_maop_c  

            from InlineInspectionCrackAnomaly f

            left join ListDomain ld on f.ILICAStatusDomainId = ld.Id
            left join ListDomain ld2 on f.ILICATypeDomainId = ld2.Id
            left join StationSeries ss on f.StationSeriesId = ss.id
            left join LineLoop ll on ss.LineLoopId = ll.Id
            left join inlineinspectionrange r on f.inlineinspectionrangeid = r.id
            left join ListDomain ld3 on r.ILIRToolDomainId = ld3.Id
            left join InlineInspection i on f.InlineInspectionId = i.Id
            left join ListDomain ld4 on i.ILICompanyDomainId = ld4.Id
            left join MLVCorrection mlv on f.StationSeriesId = mlv.StationSeriesId

            left join 
                (select ll.id [LinloopId],
                ll.LineName,
                ps.EffectiveStartDate,
                ps.PipeInserviceDate,
                ps.PipeOutsideDiameter,
                ps.PipeWallThickness,
                ps.PipeGrade,
                ps.PipeToughness,
                (ps.BeginStationNum*mlv1.MultiplierNum+mlv1.FactorNum) [begin_ps_c],
                (ps.EndStationNum*mlv2.MultiplierNum+mlv2.FactorNum) [end_ps_c]
                from PipeSegment ps
                join StationSeries ss on ps.BeginStationSeriesId = ss.id
                join LineLoop ll on ss.LineLoopId = ll.Id
                join MLVCorrection mlv1 on ps.BeginStationSeriesId = mlv1.StationSeriesId
                join MLVCorrection mlv2 on ps.EndStationSeriesId = mlv2.StationSeriesId
                where ps.EffectiveEndDate is null
                ) a on ((f.StationNum*mlv.MultiplierNum+mlv.FactorNum) between a.[begin_ps_c] and a.[end_ps_c]) and a.LinloopId = ll.id

            left join 
                (select ll.id [LinloopId],
                ll.LineName,
                maop.EffectiveEndDate,
                maop.BeginStationSeriesId,
                maop.MaxAllowablePressure [MAOP_kPa],
                maop.BeginStationNum*mlv1.MultiplierNum+mlv1.FactorNum [begin_maop_c],
                maop.EndStationNum*mlv2.MultiplierNum+mlv2.FactorNum [end_maop_c]
                from maop maop
                join StationSeries ss on ss.id = maop.BeginStationSeriesId
                join LineLoop ll on ss.LineLoopId = ll.Id
                join MLVCorrection mlv1 on maop.BeginStationSeriesId = mlv1.StationSeriesId
                join MLVCorrection mlv2 on maop.EndStationSeriesId = mlv2.StationSeriesId
                where maop.EffectiveEndDate is null
                ) b on ((f.StationNum*mlv.MultiplierNum+mlv.FactorNum) between b.[begin_maop_c] and b.[end_maop_c]) and b.[LinloopId] = ll.id 

            left join
                (select ROW_NUMBER() over(partition by ff.InlineInspectionRangeId order by ff.id asc)  [RN],
                ff.id,
                ff.InlineInspectionRangeId
                from InlineInspectionCrackAnomaly ff
                )  c on f.Id = c.Id and f.InlineInspectionRangeId = c.InlineInspectionRangeId

            where f.InlineInspectionRangeId = {ILIRID} and (c.RN between {start} and {start+amt})
            order by chainage_m asc"""
    
    return sqlserver_sql(query)

In [6]:
def get_ncds_for_poe(ILIRID, start=1, amt=999999999):
    query = f"""select
            c.RN,
            ll.linename [line],

            ld4.code [vendor], 

            format(r.ILIRStartDate,'yyyy-MM-dd') [ILIRStartDate],
            ld3.code [tool],

            f.ILINCDFeatureNumber [FeatureID],
            ld.code [status], 
            ld2.code [type],
            (f.StationNum*mlv.MultiplierNum+mlv.FactorNum) [chainage_m], 
            f.ILINCDSurfaceInd, 
            f.ILINCDDepthPct [depth_fraction], 
            f.ILINCDLength [length_mm], 
            f.ILINCDWidth [width_mm], 

            format(a.PipeInserviceDate,'yyyy-MM-dd') [install_date],
            a.PipeOutsideDiameter [OD_inch],
            a.PipeWallThickness [WT_mm],
            a.PipeGrade [grade_MPa],
            a.PipeToughness [toughness_J],
            a.[begin_ps_c],
            a.[end_ps_c],

            b.MAOP_kPa,
            b.begin_maop_c,
            b.end_maop_c  

            from InlineInspectionNonCrsnDfct f

            left join ListDomain ld on f.ILINCDStatusDomainId = ld.Id
            left join ListDomain ld2 on f.ILINCDTypeDomainId = ld2.Id
            left join StationSeries ss on f.StationSeriesId = ss.id
            left join LineLoop ll on ss.LineLoopId = ll.Id
            left join inlineinspectionrange r on f.inlineinspectionrangeid = r.id
            left join ListDomain ld3 on r.ILIRToolDomainId = ld3.Id
            left join InlineInspection i on f.InlineInspectionId = i.Id
            left join ListDomain ld4 on i.ILICompanyDomainId = ld4.Id
            left join MLVCorrection mlv on f.StationSeriesId = mlv.StationSeriesId

            left join 
                (select ll.id [LinloopId],
                ll.LineName,
                ps.EffectiveStartDate,
                ps.PipeInserviceDate,
                ps.PipeOutsideDiameter,
                ps.PipeWallThickness,
                ps.PipeGrade,
                ps.PipeToughness,
                (ps.BeginStationNum*mlv1.MultiplierNum+mlv1.FactorNum) [begin_ps_c],
                (ps.EndStationNum*mlv2.MultiplierNum+mlv2.FactorNum) [end_ps_c]
                from PipeSegment ps
                join StationSeries ss on ps.BeginStationSeriesId = ss.id
                join LineLoop ll on ss.LineLoopId = ll.Id
                join MLVCorrection mlv1 on ps.BeginStationSeriesId = mlv1.StationSeriesId
                join MLVCorrection mlv2 on ps.EndStationSeriesId = mlv2.StationSeriesId
                where ps.EffectiveEndDate is null
                ) a on ((f.StationNum*mlv.MultiplierNum+mlv.FactorNum) between a.[begin_ps_c] and a.[end_ps_c]) and a.LinloopId = ll.id

            left join 
                (select ll.id [LinloopId],
                ll.LineName,
                maop.EffectiveEndDate,
                maop.BeginStationSeriesId,
                maop.MaxAllowablePressure [MAOP_kPa],
                maop.BeginStationNum*mlv1.MultiplierNum+mlv1.FactorNum [begin_maop_c],
                maop.EndStationNum*mlv2.MultiplierNum+mlv2.FactorNum [end_maop_c]
                from maop maop
                join StationSeries ss on ss.id = maop.BeginStationSeriesId
                join LineLoop ll on ss.LineLoopId = ll.Id
                join MLVCorrection mlv1 on maop.BeginStationSeriesId = mlv1.StationSeriesId
                join MLVCorrection mlv2 on maop.EndStationSeriesId = mlv2.StationSeriesId
                where maop.EffectiveEndDate is null
                ) b on ((f.StationNum*mlv.MultiplierNum+mlv.FactorNum) between b.[begin_maop_c] and b.[end_maop_c]) and b.[LinloopId] = ll.id 

            left join
                (select ROW_NUMBER() over(partition by ff.InlineInspectionRangeId order by ff.id asc)  [RN],
                ff.id,
                ff.InlineInspectionRangeId
                from InlineInspectionNonCrsnDfct ff
                )  c on f.Id = c.Id and f.InlineInspectionRangeId = c.InlineInspectionRangeId

            where f.InlineInspectionRangeId = {ILIRID} and (c.RN between {start} and {start+amt})
            order by chainage_m asc"""
    
    return sqlserver_sql(query)

# Analysis

In [29]:
sqlserver_sql("""select ll.linename, ld.code, c.count_f, a.count_ca, b.count_ncd, r.* from inlineinspectionrange r
                join listdomain ld on r.ilirtooldomainid = ld.id
                left join (select ca.inlineinspectionrangeid, count(ca.id) [count_ca] from inlineinspectioncrackanomaly ca
                        join listdomain ld on ca.ilicastatusdomainid = ld.id
                        --where ld.code = 'Active'
                        group by ca.inlineinspectionrangeid) a on r.id = a.inlineinspectionrangeid
                left join (select ncd.inlineinspectionrangeid, count(ncd.id) [count_ncd] from inlineinspectionnoncrsndfct ncd
                        join listdomain ld on ncd.ilincdstatusdomainid = ld.id
                        join listdomain ld2 on ncd.ilincdtypedomainid = ld2.id
                        --where ld.code = 'Active'
                        where ld2.code in ('SWA-AFD','SWA-EMAT')
                        group by ncd.inlineinspectionrangeid) b on r.id = b.inlineinspectionrangeid
                left join (select f.inlineinspectionrangeid, count(f.id) [count_f] from inlineinspectionfeature f
                        join listdomain ld on f.ilifstatusdomainid = ld.id
                        join listdomain ld2 on f.iliftypedomainid = ld2.id
                        --where ld.code = 'Active'
                        group by f.inlineinspectionrangeid) c on r.id = c.inlineinspectionrangeid
                join stationseries ss on r.beginstationseriesid = ss.id
                join lineloop ll on ss.lineloopid = ll.id
                --where ld.code in ('USCD','EMAT','MFL/CMFL/EMAT/Caliper')""").sort_values(['linename','ILIRStartDate'], ascending=[True,False])\
.assign(length = lambda x: x.EndStationNum - x.BeginStationNum)[['linename','code','count_f','count_ca','count_ncd','length','ILIRStartDate','Id']]\
.query("linename.str.contains('border-burlington',case=False)")

linename          code  count_f  count_ca  \
33   PUGET BORDER-BURLINGTON 20IN KM           AFD    260.0       8.0   
112  PUGET BORDER-BURLINGTON 20IN KM      MFL_MDM2   1384.0       NaN   
61   PUGET BORDER-BURLINGTON 20IN KM          EMAT      NaN      12.0   
62   PUGET BORDER-BURLINGTON 20IN KM           AFD    108.0       NaN   
10   PUGET BORDER-BURLINGTON 20IN KM           MFL   4890.0       NaN   
17   PUGET BORDER-BURLINGTON 20IN KM           MFL   5586.0       NaN   
2    PUGET BORDER-BURLINGTON 20IN KM  Caliper Tool      NaN       NaN   
3    PUGET BORDER-BURLINGTON 20IN KM  Caliper Tool      NaN       NaN   

     count_ncd     length ILIRStartDate     Id  
33         NaN  68964.585    2018-05-15  486.0  
112        NaN  68957.881    2016-08-16  447.0  
61         NaN  68979.875    2015-08-01  368.0  
62         NaN  68979.875    2013-05-01  409.0  
10         NaN  68979.875    2012-04-13  413.0  
17         NaN  68955.429    2007-07-18  159.0  
2          NaN  68955.429    2004-08-19  157.0  
3          NaN  69262.365    2004-08-19  158.0

In [11]:
cracks = get_cracks_for_poe(624)

In [8]:
# cracks.query("chainage_m.between(819205.4,819455.297)",inplace=True)
# cracks.query("type.isin(['GCR - USCD','CRC - USCD'])",inplace=True)
cracks['vendor'] = 'Rosen3'

In [9]:
cracks.shape

(11, 23)

In [182]:
del scc

In [19]:
config = dict(iterations = 1_000_000_000,
                weibull_shape=2.,
              weibull_scale=0.26,
)
scc = POE.MonteCarlo('SCC', config=config)
scc.df = cracks.iloc[:,:].copy()
# scc.df.select_dtypes(np.float) =  scc.df.select_dtypes(np.float).astype(np.float32)
scc.process_dates()
scc.run(split_iterations=True,iterations_split=1000)
scc.result

Model: SCC POE Simulation
Count of anomalies: 1
Iterations: 1,000,000,000
Date of analysis: 2020-07-16
Weibull Shape: 2.0
Weibull Scale: 0.26
Leak threshold modifier: 0.8
Rupture threshold modifier: 1.0
Aggregated POE for these features is 8.999999967507222e-09.

Calculation took 2573.4683 seconds.


FeatureID  fail_count  iterations  rupture_count  leak_count  nan  PDP_frac  \
0     LIW 1           9  1000000000              0           9    0      0.15   

   clength           POE         POE_l  POE_r  1-POE  
0     34.0  9.000000e-09  9.000000e-09    0.0    1.0

In [20]:
scc.merge_result('FeatureID').sort_values('chainage_m').to_clipboard()#.style.format({'POE':'{:.4e}'})

In [28]:
scc.df.iloc[0].to_clipboard()

In [10]:
lofs = get_ncds_for_poe(625)

In [13]:
lofs.query('type.str.contains("SWA")',inplace=True)
lofs['tool'] = 'AFD'

In [25]:
lofs[~lofs['depth_fraction'].isnull()]#.query("chainage_m.between(748816,749064.65)")

RN                            line vendor ILIRStartDate tool  \
62      63  TMPL DARFIELD-BLACK PINES 24IN   None    2017-11-28  AFD   
84      85  TMPL DARFIELD-BLACK PINES 24IN   None    2017-11-28  AFD   
312    313  TMPL DARFIELD-BLACK PINES 24IN   None    2017-11-28  AFD   
484    483  TMPL DARFIELD-BLACK PINES 24IN   None    2017-11-28  AFD   
530    529  TMPL DARFIELD-BLACK PINES 24IN   None    2017-11-28  AFD   
...    ...                             ...    ...           ...  ...   
1398  1394  TMPL DARFIELD-BLACK PINES 24IN   None    2017-11-28  AFD   
1435  1431  TMPL DARFIELD-BLACK PINES 24IN   None    2017-11-28  AFD   
1466  1462  TMPL DARFIELD-BLACK PINES 24IN   None    2017-11-28  AFD   
1470  1466  TMPL DARFIELD-BLACK PINES 24IN   None    2017-11-28  AFD   
1473  1469  TMPL DARFIELD-BLACK PINES 24IN   None    2017-11-28  AFD   

     FeatureID  status     type  chainage_m ILINCDSurfaceInd  depth_fraction  \
62     SWA 816  Active  SWA-AFD  747166.713                I            0.11   
84     SWA 799  Active  SWA-AFD  748147.171                E            0.12   
312    SWA 577  Active  SWA-AFD  750515.946                I            0.12   
484    SWA 504  Active  SWA-AFD  758133.742                E            0.10   
530    SWA 498  Active  SWA-AFD  759168.700                E            0.10   
...        ...     ...      ...         ...              ...             ...   
1398    SWA 47  Active  SWA-AFD  779781.410                E            0.23   
1435    SWA 25  Active  SWA-AFD  781693.530                E            0.11   
1466    SWA 10  Active  SWA-AFD  783093.195                I            0.10   
1470     SWA 7  Active  SWA-AFD  783529.190                I            0.10   
1473     SWA 5  Active  SWA-AFD  783664.076                E            0.10   

      length_mm  width_mm install_date  OD_inch  WT_mm  grade_MPa  \
62         28.0      16.0   1953-01-01     24.0   7.92      359.0   
84         69.0      16.0   1953-01-01     24.0   7.92      359.0   
312        47.0      13.0   1953-01-01     24.0   7.92      359.0   
484        42.0      13.0   1953-01-01     24.0   7.92      359.0   
530        79.0      15.0   1953-01-01     24.0   7.92      359.0   
...         ...       ...          ...      ...    ...        ...   
1398      400.0      24.0   1953-01-01     24.0   7.92      359.0   
1435       60.0      16.0   1953-01-01     24.0   7.92      359.0   
1466       37.0      14.0   1953-01-01     24.0   7.92      359.0   
1470       30.0      12.0   1953-01-01     24.0   7.92      359.0   
1473       96.0      14.0   1953-01-01     24.0   7.92      359.0   

      toughness_J  begin_ps_c    end_ps_c   MAOP_kPa  begin_maop_c  end_maop_c  
62           21.7  743740.942  770390.768  6716.4094    743740.942  756269.204  
84           21.7  743740.942  770390.768  6716.4094    743740.942  756269.204  
312          21.7  743740.942  770390.768  6716.4094    743740.942  756269.204  
484          21.7  743740.942  770390.768  4940.6390    758122.099  758134.214  
530          21.7  743740.942  770390.768  4858.4336    759166.136  759176.105  
...           ...         ...         ...        ...           ...         ...  
1398         21.7  770400.163  783858.250  6716.4094    770400.163  783858.250  
1435         21.7  770400.163  783858.250  6716.4094    770400.163  783858.250  
1466         21.7  770400.163  783858.250  6716.4094    770400.163  783858.250  
1470         21.7  770400.163  783858.250  6716.4094    770400.163  783858.250  
1473         21.7  770400.163  783858.250  6716.4094    770400.163  783858.250  

[73 rows x 23 columns]

In [45]:
del md

In [44]:
config = dict(iterations = 100_000_000,
                weibull_shape=2.,
              weibull_scale=0.26,
)
md = POE.MonteCarlo('MD', config=config)
md.df = lofs.query("chainage_m.between(440058.437,440308.085)").copy()
# scc.df.select_dtypes(np.float) =  scc.df.select_dtypes(np.float).astype(np.float32)
md.process_dates()
md.run(split_iterations=True,iterations_split=100)
md.result

C:\Users\armando_borjas\Documents\Python\Calculators\general_POE.py:1087: RuntimeWarning: invalid value encountered in less
  "sdPDP": np.where(WTm*cPDP < 4, 0.780, 3.120),  # absolute mm
C:\Users\armando_borjas\Documents\Python\Calculators\general_POE.py:293: RuntimeWarning: invalid value encountered in less_equal
  ruptures = fail_pressure <= thresh*operating_pressure
C:\Users\armando_borjas\Documents\Python\Calculators\general_POE.py:313: RuntimeWarning: invalid value encountered in greater_equal
  leaks = fD >= thresh * wt


KeyboardInterrupt: 

In [43]:
md.merge_result('FeatureID').sort_values('chainage_m').to_clipboard()#.style.format({'POE':'{:.4e}'})

In [52]:
mls = get_features_for_poe(447)
mls.query("status == 'Active'",inplace=True)

In [62]:
mls.query("chainage_m.between(56879.576,56903.448)").describe(include='all')

RN                             line vendor ILIRStartDate  \
count    15.000000                               15     15            15   
unique         NaN                                1      1             1   
top            NaN  PUGET BORDER-BURLINGTON 20IN KM  Rosen    2016-08-16   
freq           NaN                               15     15            15   
mean    375.400000                              NaN    NaN           NaN   
std       7.238784                              NaN    NaN           NaN   
min     360.000000                              NaN    NaN           NaN   
25%     372.500000                              NaN    NaN           NaN   
50%     377.000000                              NaN    NaN           NaN   
75%     380.500000                              NaN    NaN           NaN   
max     384.000000                              NaN    NaN           NaN   

            tool FeatureID  status        type    chainage_m ILIFSurfaceInd  \
count         15        15      15          15     15.000000             15   
unique         1        15       1           1           NaN              1   
top     MFL_MDM2  DMA 1138  Active  Metal Loss           NaN              E   
freq          15         1      15          15           NaN             15   
mean         NaN       NaN     NaN         NaN  56887.324067            NaN   
std          NaN       NaN     NaN         NaN      5.852505            NaN   
min          NaN       NaN     NaN         NaN  56880.379000            NaN   
25%          NaN       NaN     NaN         NaN  56882.959500            NaN   
50%          NaN       NaN     NaN         NaN  56885.986000            NaN   
75%          NaN       NaN     NaN         NaN  56891.538000            NaN   
max          NaN       NaN     NaN         NaN  56900.281000            NaN   

        depth_fraction  length_mm   width_mm vendor_cgr_mmpyr vendor_cgr_sd  \
count        15.000000  15.000000  15.000000                0             0   
unique             NaN        NaN        NaN                0             0   
top                NaN        NaN        NaN              NaN           NaN   
freq               NaN        NaN        NaN              NaN           NaN   
mean          0.188000  13.926067  12.877733              NaN           NaN   
std           0.158439   3.917939   3.075551              NaN           NaN   
min           0.080000   8.611000   8.458000              NaN           NaN   
25%           0.090000  10.630000  10.477500              NaN           NaN   
50%           0.110000  13.335000  11.506000              NaN           NaN   
75%           0.240000  16.903500  15.481500              NaN           NaN   
max           0.590000  20.523000  18.110000              NaN           NaN   

       install_date  OD_inch         WT_mm  grade_MPa   toughness_J  \
count            15     15.0  1.500000e+01       15.0  1.500000e+01   
unique            1      NaN           NaN        NaN           NaN   
top      1955-01-01      NaN           NaN        NaN           NaN   
freq             15      NaN           NaN        NaN           NaN   
mean            NaN     20.0  6.350000e+00      359.0  2.170000e+01   
std             NaN      0.0  1.838704e-15        0.0  3.677408e-15   
min             NaN     20.0  6.350000e+00      359.0  2.170000e+01   
25%             NaN     20.0  6.350000e+00      359.0  2.170000e+01   
50%             NaN     20.0  6.350000e+00      359.0  2.170000e+01   
75%             NaN     20.0  6.350000e+00      359.0  2.170000e+01   
max             NaN     20.0  6.350000e+00      359.0  2.170000e+01   

        begin_ps_c      end_ps_c   MAOP_kPa  begin_maop_c    end_maop_c  
count       15.000  1.500000e+01    15.0000        15.000  1.500000e+01  
unique         NaN           NaN        NaN           NaN           NaN  
top            NaN           NaN        NaN           NaN           NaN  
freq           NaN           NaN        NaN           Na

In [54]:
mls['vendor']='Rosen'

In [ ]:
del corr

In [60]:
def cgr(**kwargs):
#     alpha = kwargs['vcgr']/kwargs['vcgr_sd']
#     return np.where(alpha > 3.0,
#                     np.maximum(0, norm.ppf(kwargs['random'], loc=kwargs['vcgr'], scale=kwargs['vcgr_sd']))/25.4,
#                     POE.cgr_weibull(kwargs['random'], 1.24595, 0.107359) / 25.4)
    return POE.cgr_weibull(kwargs['random'], 1.24595, 0.107359) / 25.4
    # return 12.5/1000.
#     return kwargs['depth']/kwargs['ILI_age']

config = dict(iterations=1_000_000,
              leak_thresh=0.96,
              rupt_thresh=1.1,
              cgr=cgr)

corr = POE.MonteCarlo('CORR', config=config)
corr.df = mls.query("chainage_m.between(56879.576,56903.448)").copy()
# scc.df.select_dtypes(np.float) =  scc.df.select_dtypes(np.float).astype(np.float32)
corr.process_dates()
corr.run()#split_iterations=True,iterations_split=10)
corr.result

Model: CORR POE Simulation
Count of anomalies: 15
Iterations: 1,000,000
Date of analysis: 2020-07-21
Leak threshold modifier: 0.96
Rupture threshold modifier: 1.1
Aggregated POE for these features is 0.0013789728199999907.

Calculation took 21.1020 seconds.


FeatureID  fail_count  iterations  rupture_count  leak_count  nan  \
0   DMA 1114        1359     1000000             63        1341  0.0   
1   DMA 1115           0     1000000              0           0  0.0   
2   DMA 1116           0     1000000              0           0  0.0   
3   DMA 1117           0     1000000              0           0  0.0   
4   DMA 1118          20     1000000              1          20  0.0   
5   DMA 1119           0     1000000              0           0  0.0   
6   DMA 1120           0     1000000              0           0  0.0   
7   DMA 1121           0     1000000              0           0  0.0   
8   DMA 1122           0     1000000              0           0  0.0   
9   DMA 1124           0     1000000              0           0  0.0   
10  DMA 1125           0     1000000              0           0  0.0   
11  DMA 1126           0     1000000              0           0  0.0   
12  DMA 1127           0     1000000              0           0  0.0   
13  DMA 1137           0     1000000              0           0  0.0   
14  DMA 1138           0     1000000              0           0  0.0   

    PDP_frac  flength       POE     POE_l     POE_r     1-POE  
0       0.59   13.081  0.001359  0.001341  0.000063  0.998641  
1       0.33   11.405  0.000000  0.000000  0.000000  1.000000  
2       0.11   10.541  0.000000  0.000000  0.000000  1.000000  
3       0.09    9.957  0.000000  0.000000  0.000000  1.000000  
4       0.42   17.221  0.000020  0.000020  0.000001  0.999980  
5       0.11   20.523  0.000000  0.000000  0.000000  1.000000  
6       0.36    9.119  0.000000  0.000000  0.000000  1.000000  
7       0.09   16.586  0.000000  0.000000  0.000000  1.000000  
8       0.15   15.977  0.000000  0.000000  0.000000  1.000000  
9       0.08   18.593  0.000000  0.000000  0.000000  1.000000  
10      0.10   13.716  0.000000  0.000000  0.000000  1.000000  
11      0.08    8.611  0.000000  0.000000  0.000000  1.000000  
12      0.11   13.335  0.000000  0.000000  0.000000  1.000000  
13      0.08   19.507  0.000000  0.000000  0.000000  1.000000  
14      0.12   10.719  0.000000  0.000000  0.000000  1.000000

In [63]:
sqlserver_sql('''select * from virtualtable vt''').query('VirtualTableName.str.contains("SCC")')

Id         VirtualTableName  VirtualTableOverlapFlg  \
0      1.0                    SCCPs                     0.0   
26    27.0                SCCDADate                     0.0   
27    28.0               SCCHistory                     0.0   
60    75.0  SCCReassessmentInterval                     0.0   
87   102.0                   SCCPOI                     0.0   
88   103.0                   SCCPOD                     0.0   
89   104.0       SCCCrackGrowthRate                     0.0   
202  222.0       SCCNNHistoryStatus                     0.0   

               VirtualTableComment                  LastModByUserId  \
0                       DO NOT USE  DYNAMIC_RISK\Aravind_Peddaboina   
26                     SCC DA Date       DYNAMIC_RISK\martin_friese   
27                     SCC History       DYNAMIC_RISK\martin_friese   
60       SCC Reassessment Interval                                    
87                      DO NOT USE  DYNAMIC_RISK\Aravind_Peddaboina   
88                      DO NOT USE  DYNAMIC_RISK\Aravind_Peddaboina   
89   SCC Crack Growth Rate (mm/yr)         DYNAMIC_RISK\winston_lee   
202             SCC History Status  DYNAMIC_RISK\Aravind_Peddaboina   

            LastModDateTime  
0   2018-06-14 09:05:23.660  
26  2009-06-09 09:03:24.143  
27  2009-06-09 09:03:24.143  
60  2012-06-27 08:09:59.000  
87  2018-06-14 09:05:23.660  
88  2018-06-14 09:05:23.660  
89  2015-07-07 07:42:45.667  
202 2018-06-18 09:43:39.020

In [ ]:
sqlserver_sql('''select * from virtualtabledatanum vtn
                join virtualtablestationing vts on vtn.virtualtablestationingid = vts.virtualtablestationingid and vts.virtualtableid = vtn.virtualtableid''')

## Resident Damage

In [2]:
%cd ..\Calculators

C:\Users\armando_borjas\Documents\Python\Calculators


In [3]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is FE48-FC0C

 Directory of C:\Users\armando_borjas\Documents\Python\Calculators

2020-08-31  04:02 PM    <DIR>          .
2020-08-31  04:02 PM    <DIR>          ..
2020-09-02  09:07 AM    <DIR>          __pycache__
2020-08-31  11:47 AM    <DIR>          Archive
2018-07-05  02:30 PM             3,430 cgr_excel.py
2018-08-27  01:00 PM             3,250 crack_gr_excel.py
2018-09-24  11:36 AM             8,583 cscc_bulk_offline.xlsx
2018-10-01  01:06 PM             8,640 CSCC_POE_numpy.py
2019-01-22  02:20 PM             4,351 derive_length_modified_lnsec_optimized.py
2019-01-22  10:25 AM             2,833 derive_length_ng18_optimized.py
2019-01-22  02:52 PM             4,164 derive_length_original_lnsec_optimized.py
2018-09-24  02:06 PM             1,370 excel_merger.py
2019-02-07  11:10 AM         1,003,356 feature_list.csv
2020-08-31  11:45 AM             5,240 FTS sample_of_inputs for Armando Open Air  08312020.csv
2020-08-31  11:

In [9]:
config = dict(iterations=1_000_000,
#cgr=cgr
)
# corr = MonteCarlo('CORR', config=config)

# corr.get_data('sample_of_inputs.csv')
# corr.df = corr.df.iloc[0:10]
# corr.run()


# mcrun = MonteCarlo('CORR', config=config)
# mcrun.get_data('sample_of_inputs.csv')
# mcrun.df = mcrun.df.iloc[0:10]
# mcrun.special_run()

res = POE.MonteCarlo('RD', config=config)
res.get_data('sample_of_inputs.csv')
res.run()
# res.merge_result('FeatureID')
res.merge_result("FeatureID")

Model: RD POE Simulation
Count of anomalies: 12
Iterations: 1,000,000
Date of analysis: 2020-09-02
Leak threshold modifier: 0.8
Rupture threshold modifier: 1.0
Aggregated POE for these features is 0.631446844605706.

Calculation took 15.5335 seconds.


FeatureID  fail_count  leak_count  rupture_count  iterations  NPS_Frac  \
0     1527296           0           0              0     1000000     0.005   
1     1527304           0           0              0     1000000     0.010   
2     1531955           0           0              0     1000000     0.015   
3     1527230          72          72             72     1000000     0.020   
4     1527293        3872        3872           3872     1000000     0.025   
5     1527309       45101       45101          45101     1000000     0.030   
6     1527300      132620      132620         132620     1000000     0.035   
7     1532384      163674      163674         163674     1000000     0.040   
8       32724      155957      155957         155957     1000000     0.045   
9     1530568      146797      146797         146797     1000000     0.050   
10    1527246      140119      140119         140119     1000000     0.055   
11    1531611      137382      137382         137382     1000000     0.060   

    dlength  dwidth  time_delta       POE     POE_l     POE_r     1-POE  \
0       200      34    8.670773  0.000000  0.000000  0.000000  1.000000   
1       200      34    8.670773  0.000000  0.000000  0.000000  1.000000   
2       200      34    8.670773  0.000000  0.000000  0.000000  1.000000   
3       200      34    8.670773  0.000072  0.000072  0.000072  0.999928   
4       200      34    8.670773  0.003872  0.003872  0.003872  0.996128   
5       200      34    8.670773  0.045101  0.045101  0.045101  0.954899   
6       200      34    8.670773  0.132620  0.132620  0.132620  0.867380   
7       200      34    8.670773  0.163674  0.163674  0.163674  0.836326   
8       200      34    8.670773  0.155957  0.155957  0.155957  0.844043   
9       200      34    8.670773  0.146797  0.146797  0.146797  0.853203   
10      200      34    8.670773  0.140119  0.140119  0.140119  0.859881   
11      200      34    8.670773  0.137382  0.137382  0.137382  0.862618   

      line vendor     tool ILIRStartDate  status  type ILIFSurfaceInd  \
0   Line 1  Rosen  Caliper    2012-01-01  Active  dent              E   
1   Line 1  Rosen  Caliper    2012-01-01  Active  dent              I   
2   Line 1  Rosen  Caliper    2012-01-01  Active  dent              E   
3   Line 1  Rosen  Caliper    2012-01-01  Active  dent              E   
4   Line 1  Rosen  Caliper    2012-01-01  Active  dent              E   
5   Line 1  Rosen  Caliper    2012-01-01  Active  dent              E   
6   Line 1  Rosen  Caliper    2012-01-01  Active  dent              E   
7   Line 1  Rosen  Caliper    2012-01-01  Active  dent              E   
8   Line 1  Rosen  Caliper    2012-01-01  Active  dent              I   
9   Line 1  Rosen  Caliper    2012-01-01  Active  dent              E   
10  Line 1  Rosen  Caliper    2012-01-01  Active  dent              E   
11  Line 1  Rosen  Caliper    2012-01-01  Active  dent              E   

    chainage_m  depth_fraction  length_mm  width_mm  vendor_cgr_mmpyr  \
0            1           0.005        200        34               NaN   
1            1           0.010        200        34               NaN   
2            1           0.015        200        34               NaN   
3            1           0.020        200        34               NaN   
4            1           0.025        200        34               NaN   
5            1           0.030        200        34               NaN   
6            1           0.035        200        34               NaN   
7            1           0.040        200        34               NaN   
8            1           0.045        200        34               NaN   
9            1           0.050        200        34               NaN   
10           1           0.055        200        34               NaN   
11           1           0.060        200        34               NaN   

    vendor_cgr_sd  OD_inch  WT_mm  grade_MPa  toughness_J install_date  \
0             NaN       24    7.6      

In [83]:
res.result[['fail_count','POE']].to_clipboard(header=False,index=False)

In [13]:
POE.nf_EPRG(609.09,7.757,659.644,201.450,50.458,29.8513,0.,89.935789,0.)

1787830.8354204996

In [20]:
((datetime.date.today()-res.df['ILIRStartDate']).dt.days/365.25)*res.df['AESC']

0     86652.977413
1     86652.977413
2     86652.977413
3     86652.977413
4     86652.977413
5     86652.977413
6     86652.977413
7     86652.977413
8     86652.977413
9     86652.977413
10    86652.977413
11    86652.977413
dtype: float64